Assignment 9 
Julia Ruiter

In the diabetes.csv dataset, the column label represents the class label of the record (268 of 768 are 1 – diabetic, the others are 0). You have to apply three different classification techniques to predict the label of each record (You may use SVM, Random Forests and Neural Networks).

In [81]:
# libraries
# dataframes
import numpy as np
import pandas as pd

# linear models
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

# import specific log regression libraries -- not used in this example
import random
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score

# random forest models
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

# for svm
from sklearn.svm import SVC

# for the confusion matrix outputs
from sklearn.metrics import confusion_matrix   

In [3]:
# import csv file and create df
diabetes_df  = pd.read_csv("diabetes.csv")  # 768 tuples
diabetes_df = diabetes_df.sample(frac=1).reset_index(drop=True)  # randomly reorder df rows and delete initial indexes
diabetes_df.head()  # preview df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,5,155,84,44,545,38.7,0.619,34,0
1,4,99,76,15,51,23.2,0.223,21,0
2,6,111,64,39,0,34.2,0.260,24,0
3,1,97,64,19,82,18.2,0.299,21,0
4,4,141,74,0,0,27.6,0.244,40,0


In [22]:
# test and training sets

diabetes_test = diabetes_df.sample(frac=0.25,random_state=42)  #random state is a seed value, put 25% in test set
diabetes_training = diabetes_df.drop(diabetes_test.index)   # the rest

len(diabetes_training)   # check that 75% = 576 tuples like in R code ==> yes


# set up the subsets; y is 'Outcome'
x_train = diabetes_training.loc[:, diabetes_training.columns != 'Outcome']
y_train = diabetes_training.loc[:, diabetes_training.columns == 'Outcome']

x_test = diabetes_test.loc[:, diabetes_test.columns != 'Outcome']
y_test = diabetes_test.loc[:, diabetes_test.columns == 'Outcome']

y_test_df = pd.DataFrame(y_test)  # for augmenting results later

METHOD 1: Linear Regression

Exercise 2 - Linear Regression
Train the model using the training data.

In [62]:
linear_regressor = LinearRegression() # create object for the class
linear_regressor.fit(x_train, y_train) # perform linear regression

LinearRegression()

Exercise 3 - Linear Regression
Use the model to predict the labels of the records in the test data.

In [68]:
y_predicted_linear = linear_regressor.predict(x_test)   # make predictions
y_linear = pd.DataFrame(y_predicted_linear, columns=['pred'])  # convert prediction to dataframe
y_linear

# WILL NOT APPEND ==> can use direct values in next step
#y_test_df['y_linear'] = y_predicted_linear # append column with predicted values
#y_test_df  # output actual versus predicted values

,pred
0,0.663746
1,0.527846
2,0.124294
3,0.372113
4,0.504868
...,...
187,0.725628
188,0.748765
189,0.219524
190,-0.198975


Exercise 4 - Linear Regression
Compute and display the confusion matrix.

In [78]:
# set threshold value for predictions
linear_cutoff = 0.75

#y_linear.pred.apply(lambda x: x/x if x >= linear_cutoff else x*0)

# reassign values based on threshold
y_linear[y_linear['pred'] >= linear_cutoff] = 1
y_linear[y_linear['pred'] < linear_cutoff] = 0

#y_linear
confusion_matrix(y_test_df.iloc[:, 0], y_linear.iloc[:, 0])
# not very good -- similar to r output

array([[108,  16],
       [ 30,  38]], dtype=int64)

METHOD 2: Random Forests

Exercise 2 - Random Forests
Train the model using the training data.

In [92]:
forest_regressor = RandomForestRegressor(n_estimators = 5, random_state = 42)   # create object for the class
forest_regressor.fit(x_train, y_train.values.ravel())     # perform forest regression
# ravel() is needed to make sure that the y is a numeric output

RandomForestRegressor(n_estimators=5, random_state=42)

Exercise 3 - Random Forests
Use the model to predict the labels of the records in the test data.

In [93]:
y_predicted_forest = forest_regressor.predict(x_test)   # make predictions
y_forest = pd.DataFrame(y_predicted_forest, columns=['pred'])  # convert prediction to dataframe
y_forest

,pred
0,0.6
1,0.8
2,0.0
3,1.0
4,0.6
...,...
187,0.6
188,0.8
189,0.0
190,0.0


Exercise 4 - Random Forests
Compute and display the confusion matrix.

In [99]:
# set threshold value for predictions
forest_cutoff = 0.8

#y_linear.pred.apply(lambda x: x/x if x >= linear_cutoff else x*0)

# reassign values based on threshold
y_forest[y_forest['pred'] >= forest_cutoff] = 1
y_forest[y_forest['pred'] < forest_cutoff] = 0

#y_linear
confusion_matrix(y_test_df.iloc[:, 0], y_forest.iloc[:, 0])
# not very good -- somehow much worse than r output, even when I change the threshold cutoff value

array([[115,   9],
       [ 35,  33]], dtype=int64)

METHOD 3: SVM

Exercise 2 - SVM
Train the model using the training data.

In [101]:
svclassifier = SVC(kernel='linear')

#svm_regressor = RandomForestRegressor(n_estimators = 5, random_state = 42)   # create object for the class
svclassifier.fit(x_train, y_train.values.ravel())     # perform forest regression
# ravel() is needed to make sure that the y is a numeric output

SVC(kernel='linear')

Exercise 3 - SVM
Use the model to predict the labels of the records in the test data.

In [102]:
y_predicted_svm = svclassifier.predict(x_test)   # make predictions
y_svm = pd.DataFrame(y_predicted_svm, columns=['pred'])  # convert prediction to dataframe
y_svm

,pred
0,1
1,1
2,0
3,0
4,0
...,...
187,1
188,1
189,0
190,0


Exercise 4 - SVM
Compute and display the confusion matrix.

In [103]:
# no need to round values here -- already classified as 1 or 0

confusion_matrix(y_test_df.iloc[:, 0], y_svm.iloc[:, 0])
# also not very good -- but about on par with the other 2 methods

array([[108,  16],
       [ 31,  37]], dtype=int64)